# Trabajo Final

###  CLASIFICACION CON GMMs

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

### Integrantes:

Nombre: David de Jesus Yepes Isaza

Cédula: 1020455542

Nombre: Rodrigo Antonio Porras Martinez

Cédula: 1039459720

### Clasificación en el dataset "Heart Disease Data Set" con GaussianMixture

La base de datos con las que se trabaja en el presente artículo fue extraída de  Cleveland Clinic Foundation y fue creada por V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.

El set de datos de de Cleveland consta de 303 instancias, la variable de salida o etiqueta son 0 para la ausencia y los valores 1, 2, 3 y 4 para la presencia de enfermedades. 
 
El 54% de las instancias tienen ausencia de enfermedades y 46% tienen presencia de enfermedades del corazón. Para simplificar el problema se analiza como un problema biclase donde  “0” son personas sanas y “1” que corresponde a (1,2,3,4) son personas enfermas.

El objetivo es según las características de cada paciente identificar cuál de ellos presenta presencia de enfermedades y cuáles no.

La base de datos contiene 303 muestras con las 14 características siguientes:
1. N° 3 (edad)   
2. N° 4 (sexo) 
3. N° 9 (cp) tipo de dolor en el pecho
        Valor 1: angina típica
        Valor 2: angina atípica
        Valor 3: dolor no anginal
        Valor 4: asintomático  
4. N° 10 (trestbps) presión arterial en reposo
5. N° 12 (chol) colesterol sérico en mg / dl (miligramo por decilitro)
6. N° 16 (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
7. N° 19 (restecg)  resultados electrocardiográficos en reposo:
        Valor 0: normal
        Valor 1: tiene anormalidad de onda ST-T (inversiones de onda T y / o ST  elevación o depresión de > 0.05 mV)
        Valor 2: muestra hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes  
8. N° 32 (thalach)  frecuencia cardíaca máxima lograda
9. N° 38 (exang) angina inducida por el ejercicio (1 = sí; 0 = no)
10. N° 40 (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo
11. N° 41 (pendiente) la pendiente del segmento ST de ejercicio máximo
        Valor 1: pendiente ascendente
        Valor 2: plano
        Valor 3: downsloping
12. N° 44 (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. N° 51 (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible
14. N° 58 (num) (el atributo predicho) diagnóstico de enfermedad cardíaca (estado angiográfico de la enfermedad)
        - Valor 0: <50% de diámetro de estrechamiento
        - Valor 1:> 50% de diámetro de estrechamiento

### Importacion de Librerias y error de clasificación

In [6]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.mixture import GaussianMixture
from __future__ import division
import time
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

### Eliminacion de valores Perdidos (Missing Values)

En la base de datos algunos valores están perdidos, se pueden identificar por el valor “-9.0”. por lo que se deben limpiar.

In [7]:
#cargamos la bd de entrenamiento
db = np.loadtxt('cleveland.txt',delimiter=',')  # El delimitador es ","
print( "Dim de la base de datos original: " + str(np.shape(db)))
db = db.reshape(303,14)
DataBase = db
#La base d edatos processed.cleveland tiene valores perdidos, vamos a quitar todos esos registros de la base de datos
#Están marcados como -9.0. Donde haya un valor -9.0 eliminaremos el registro.
j = 0
for i in range(0,np.size(db,0)):
    if -9.0 == db[i,13]:
        j+=1
        DataBase = np.delete(DataBase,i,0)
    elif 1 <= db[i,13]: #Se cambia la variables de salida (1,2,3 y 4) por el valor 1 para obtener un problema biclase
        DataBase[i,13]=1
    
print ("\nHay " + str(j) + " valores perdidos en la variable de salida. :(")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(DataBase)))

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,14):
        if -9.0 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -9.0 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")

print("Dimension: "+ str(np.shape(DataBase)))
#print(DataBase)

Dim de la base de datos original: (303, 14)

Hay 0 valores perdidos en la variable de salida. :(

Dim de la base de datos sin las muestras con variable de salida perdido (303, 14)

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase
Dimension: (303, 14)


In [10]:
#cargamos la bd de entrenamiento  despues de tenerla limpia

X = DataBase[:,0:13]

Y = db[:,13]

#print (np.shape(Y))

#Normalizamos las caracterisiticas
X = normalize(X, axis=0, norm='l1')

#Para calcular el costo computacional
tiempo_i = time.time()

#Para obtener el numero de clases
n_classes = len(np.unique(Y))
#print(n_classes)

estimator = GaussianMixture(n_components=n_classes, covariance_type='full')

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)

j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    
    #Con la base de datos se realiza una inicialización de las medias de cada Gaussiana de manera supervisada
    estimator.means_init = np.array([X_train[y_train == i].mean(axis=0)
                                    for i in range(n_classes)])
   
    # Entrenar el modelo.
    estimator.fit(X_train)
    
    # Validación del modelo
    ypred = estimator.predict(X_test)

    Errores[j] = classification_error(ypred, y_test)
    tn, fp, fn, tp = confusion_matrix(y_test, ypred).ravel()
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
    
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia: "+str(np.mean(Eficiencia)))
print (("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i)) + " segundos.\n")



Error de validación: 0.42591397849462365 +/- 0.128498402787815

Sensibilidad: 0.228485803412274

Especificidad: 0.8657686495650893

Precision: 0.5875

Eficiencia: 0.5740860215053762


Tiempo total de ejecución: 0.2389240264892578 segundos.



<table>
  <tr>
    <th>Clases</th>
    <th>var-cov</th>
    <th>Sensibilidad</th>
    <th>Especificidad</th>
    <th>Precision</th>
    <th>Eficiencia</th>
    <th>Error de validación</th>
    <th>Tiempo de ejecución</th>
  </tr>
  <tr>
    <td>2</td>
    <td>Full</td>
    <td>0.33974917239623126</td>
    <td>0.7187761683620817</td>
    <td>0.5308333333333333</td>
    <td>0.5474193548387096</td>
    <td>0.3959139784946237 +/- 0.1488847817229718</td>
    <td>0.5328292846679688 segundos.</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Diag</td>
      <td>0.6047373949579833</td>
      <td>0.8497160179861417</td>
      <td>0.7811546786546787</td>
      <td>0.7330107526881721</td>
    <td>0.26698924731182794 +/- 0.06459323349538117</td>
    <td>0.06897854804992676 segundos.</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Spherical</td>
      <td>0.45680831423478485</td>
      <td>0.6493579395320881</td>
      <td>0.5291179408826467</td>
      <td>0.554516129032258</td>
    <td>0.44548387096774195 +/- 0.11675463706812943</td>
    <td>0.07297635078430176 segundos.</td>
  </tr>
</table>


Con el anterior podemos concluir que el modelo GMMs no obtiene muy buenos resultados con nuestra base de datos, el mejor resultado lo obtuvo con la matriz de covarianzas diagonal con una eficiencia de 0.73. El costo computacional no es alto, pero debemos recordar que son solo 303 muestras.